
<br>
===========================================================<br>
A demo of K-Means clustering on the handwritten digits data<br>
===========================================================<br>
In this example we compare the various initialization strategies for<br>
K-means in terms of runtime and quality of the results.<br>
As the ground truth is known here, we also apply different cluster<br>
quality metrics to judge the goodness of fit of the cluster labels to the<br>
ground truth.<br>
Cluster quality metrics evaluated (see :ref:`clustering_evaluation` for<br>
definitions and discussions of the metrics):<br>
=========== ========================================================<br>
Shorthand    full name<br>
=========== ========================================================<br>
homo         homogeneity score<br>
compl        completeness score<br>
v-meas       V measure<br>
ARI          adjusted Rand index<br>
AMI          adjusted mutual information<br>
silhouette   silhouette coefficient<br>
=========== ========================================================<br>


In [ ]:
print(__doc__)

In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [ ]:
np.random.seed(42)

In [ ]:
X_digits, y_digits = load_digits(return_X_y=True)
data = scale(X_digits)

In [ ]:
n_samples, n_features = data.shape
n_digits = len(np.unique(y_digits))
labels = y_digits

In [ ]:
sample_size = 300

In [ ]:
print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_digits, n_samples, n_features))

In [ ]:
print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

In [ ]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

In [ ]:
bench_k_means(KMeans(init='k-means++', n_clusters=n_digits, n_init=10),
              name="k-means++", data=data)

In [ ]:
bench_k_means(KMeans(init='random', n_clusters=n_digits, n_init=10),
              name="random", data=data)

in this case the seeding of the centers is deterministic, hence we run the<br>
kmeans algorithm only once with n_init=1

In [ ]:
pca = PCA(n_components=n_digits).fit(data)
bench_k_means(KMeans(init=pca.components_, n_clusters=n_digits, n_init=1),
              name="PCA-based",
              data=data)
print(82 * '_')

#############################################################################<br>
Visualize the results on PCA-reduced data

In [ ]:
reduced_data = PCA(n_components=2).fit_transform(data)
kmeans = KMeans(init='k-means++', n_clusters=n_digits, n_init=10)
kmeans.fit(reduced_data)

Step size of the mesh. Decrease to increase the quality of the VQ.

In [ ]:
h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

Plot the decision boundary. For that, we will assign a color to each

In [ ]:
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Obtain labels for each point in mesh. Use last trained model.

In [ ]:
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

Put the result into a color plot

In [ ]:
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

In [ ]:
plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()